In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import scipy.sparse as sp
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import optuna


In [2]:
# import df.csv

df = pd.read_csv('df.csv')

/tmp/ipykernel_187679/3391109823.py:3: DtypeWarning: Columns (27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df.csv')


In [3]:
df

,PlayerID,Date,Injury,TrainingID_lag_1,TrainingID_lag_2,TrainingID_lag_3,Exercise #_lag_1,Exercise #_lag_2,Exercise #_lag_3,ExerciseID_lag_1,...,Affectedperformance_roll_sum_3,Affectedperformance_roll_sum_7,Symptomscomplaints_roll_sum_3,Symptomscomplaints_roll_sum_7,RPE_roll_sum_3,RPE_roll_sum_7,Duration.player_roll_sum_3,Duration.player_roll_sum_7,Duration.exercise_roll_sum_3,Duration.exercise_roll_sum_7
0,4,2018-05-11,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2018-05-11,10.0,34.0,NaN,NaN,Exercise 3,NaN,NaN,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2018-05-11,10.0,34.0,34.0,NaN,Exercise 3,Exercise 3,NaN,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2018-05-11,10.0,34.0,34.0,34.0,Exercise 6,Exercise 3,Exercise 3,8.0,...,0.0,NaN,0.0,NaN,12.0,NaN,17892.0,NaN,1163.0,NaN
4,4,2018-05-11,10.0,34.0,34.0,34.0,Exercise 3,Exercise 6,Exercise 3,11.0,...,0.0,NaN,0.0,NaN,12.0,NaN,17892.0,NaN,1920.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245667,4,2018-09-20,NaN,NaN,140.0,140.0,NaN,Exercise 2,Exercise 3,NaN,...,2.0,6.0,2.0,6.0,14.0,39.0,8130.0,32062.0,3460.0,13548.0
245668,4,2018-09-21,7.0,NaN,NaN,140.0,NaN,NaN,Exercise 2,NaN,...,2.0,6.0,2.0,6.0,7.0,35.0,11265.0,36195.0,5400.0,17264.0
245669,4,2018-09-21,7.0,151.0,NaN,NaN,Exercise 2,NaN,NaN,133.0,...,2.0,6.0,2.0,6.0,0.0,28.0,14400.0,34995.0,7200.0,17264.0
245670,4,2018-09-22,NaN,151.0,151.0,NaN,Exercise 1,Exercise 2,NaN,82.0,...,2.0,5.0,2.0,5.0,0.0,21.0,22800.0,34995.0,13200.0,17264.0


In [4]:
# Assume your dataframe is df and your date column is 'date'

# Convert date to datetime if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Sort by date
df.sort_values('Date', inplace=True)

# Remove rows where the 'Injury' column has NA values
df = df.dropna(subset=['Injury'])

# Get unique dates
unique_dates = df['Date'].unique()

# Decide how many dates to include in each set
train_dates = int(len(unique_dates) * 0.8)

# Find the index of the last training date
last_train_idx = df[df['Date'] == unique_dates[train_dates]].index[-1]

# Now you can create your train and test sets
train = df.iloc[:last_train_idx + 1]
test = df.iloc[last_train_idx + 1:]



In [12]:
test


,PlayerID,Date,Injury,TrainingID_lag_1,TrainingID_lag_2,TrainingID_lag_3,Exercise #_lag_1,Exercise #_lag_2,Exercise #_lag_3,ExerciseID_lag_1,...,Affectedperformance_roll_sum_3,Affectedperformance_roll_sum_7,Symptomscomplaints_roll_sum_3,Symptomscomplaints_roll_sum_7,RPE_roll_sum_3,RPE_roll_sum_7,Duration.player_roll_sum_3,Duration.player_roll_sum_7,Duration.exercise_roll_sum_3,Duration.exercise_roll_sum_7
243949,4,2018-09-05,9.0,122.0,122.0,122.0,Exercise 1,Exercise 1,Exercise 1,63.0,...,0.0,0.0,0.0,0.0,28.0,56.0,28636.0,57272.0,2284.0,4568.0
243948,4,2018-09-05,9.0,122.0,122.0,122.0,Exercise 1,Exercise 1,Exercise 1,63.0,...,0.0,0.0,0.0,0.0,28.0,56.0,28636.0,57272.0,2284.0,4568.0
243947,4,2018-09-05,9.0,122.0,122.0,122.0,Exercise 1,Exercise 1,Exercise 1,63.0,...,0.0,0.0,0.0,0.0,28.0,56.0,28636.0,57272.0,2284.0,4568.0
243796,4,2018-09-05,9.0,122.0,122.0,122.0,Exercise 4,Exercise 4,Exercise 4,13.0,...,0.0,0.0,0.0,0.0,28.0,56.0,28636.0,57272.0,1576.0,3152.0
243795,4,2018-09-05,9.0,122.0,122.0,122.0,Exercise 4,Exercise 4,Exercise 4,13.0,...,0.0,0.0,0.0,0.0,28.0,56.0,28636.0,57272.0,1576.0,3152.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245663,4,2018-09-18,7.0,150.0,150.0,139.0,Exercise 2,Exercise 1,Exercise 3,132.0,...,4.0,8.0,4.0,8.0,25.0,41.0,23932.0,36200.0,10088.0,11975.0
245664,4,2018-09-18,7.0,140.0,150.0,150.0,Exercise 1,Exercise 2,Exercise 1,47.0,...,4.0,8.0,4.0,8.0,28.0,44.0,24930.0,37198.0,11864.0,14500.0
245665,4,2018-09-18,7.0,140.0,140.0,150.0,Exercise 3,Exercise 1,Exercise 2,88.0,...,4.0,8.0,4.0,8.0,28.0,47.0,20595.0,38196.0,10064.0,14808.0
245668,4,2018-09-21,7.0,NaN,NaN,140.0,NaN,NaN,Exercise 2,NaN,...,2.0,6.0,2.0,6.0,7.0,35.0,11265.0,36195.0,5400.0,17264.0


In [8]:
# Identify categorical columns excluding the date column
cat_cols = train.select_dtypes(include=['object']).columns.tolist()

# Initialize OneHotEncoder
ohe = OneHotEncoder(sparse=True, handle_unknown='ignore')

# Fit the OneHotEncoder on the training data and transform both train and test data
ohe.fit(train[cat_cols])
one_hot_encoded_train = ohe.transform(train[cat_cols])
one_hot_encoded_test = ohe.transform(test[cat_cols])

# Extract numeric data
numeric_data_train = train.drop(cat_cols + ['Date'], axis=1).values
numeric_data_test = test.drop(cat_cols + ['Date'], axis=1).values

# Combine sparse and dense data
final_data_train = sp.hstack((one_hot_encoded_train, numeric_data_train)).tocsr()
final_data_test = sp.hstack((one_hot_encoded_test, numeric_data_test)).tocsr()

# Set up for cross-validation
X = final_data_train
y = train["Injury"].values  # replace "Injury" with your target column name

# Initialize a TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_uniform('gamma', 0.0, 1.0),
        'objective': 'reg:squarederror'
    }

    mse_scores = []

    for train_index, val_index in tscv.split(X):
        X_train_cv, X_val_cv = X[train_index], X[val_index]
        y_train_cv, y_val_cv = y[train_index], y[val_index]

        model = xgb.XGBRegressor(**params)
        model.fit(X_train_cv, y_train_cv)

        y_pred = model.predict(X_val_cv)
        mse = mean_squared_error(y_val_cv, y_pred)
        mse_scores.append(mse)

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params_

final_model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
final_model.fit(X, y)

predictions = final_model.predict(final_data_test)

/home/n0rdp0l/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[I 2023-06-22 12:48:13,813] A new study created in memory with name: no-name-9f9a3fa3-a747-4c32-b412-eaca0e7bb287
/tmp/ipykernel_187679/2207171335.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
/tmp/ipykernel_187679/2207171335.py:32: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0

KeyboardInterrupt: 

In [9]:
# Extract hyperparameters from trial 4
best_params = {'learning_rate': 0.4033386560764858, 
               'max_depth': 6, 
               'n_estimators': 453, 
               'subsample': 0.8280774098649553, 
               'colsample_bytree': 0.9340110099277303, 
               'gamma': 0.8680536696218003,
               'objective': 'reg:squarederror'}

# Train the model with the best parameters on the entire training data
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X, y)

# Prepare the test data
X_test = final_data_test
y_test = test["Injury"].values  # replace "Injury" with your target column name

# Make predictions on the test data
predictions_test = final_model.predict(X_test)

# Calculate mean squared error on the test data
mse_test = mean_squared_error(y_test, predictions_test)

print("Mean squared error on the test data: ", mse_test)


Mean squared error on the test data:  6.29169313312704e-06
